# Вебинар 4. Домашнее задание

Само домашнее задание нахожится в конце ноутбука

---

## Задание 1. Напишите свою функцию prefilter_items

In [1]:
# на гпу только эта версия работает
!pip install implicit==0.4.0

     |████████████████████████████████| 1.1MB 2.8MB/s 
  Created wheel for implicit: filename=implicit-0.4.0-cp36-cp36m-linux_x86_64.whl size=3390278 sha256=405fe2793e8357e11b52b70b76f2981d08a13ef07c5bf3a8ea3cb276918ce7ad
  Stored in directory: /root/.cache/pip/wheels/fd/d1/66/84e83191d9f626c521271023e4ef83e634e3b8c154f2c4b2da
Successfully built implicit


In [2]:
def precision_at_k(recommended_list, bought_list, k=5):
    import numpy as np
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)

    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]

    flags = np.isin(bought_list, recommended_list)

    precision = flags.sum() / len(recommended_list)

    return precision

In [3]:
def recall_at_k(recommended_list, bought_list, k=5):
    import numpy as np
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    recommended_list = recommended_list[:k]
    flags = np.isin(bought_list, recommended_list)
    recall = flags.sum() / len(bought_list)

    return recall

In [4]:
def prefilter_items(data, take_n_popular=5000):
    """Предфильтрация товаров"""
    
    # 1. Удаление товаров, со средней ценой < 1$
    data.loc[data['sales_value'] < 1.0, ['item_id']] = 999999
    
    # 2. Удаление товаров со соедней ценой > 30$
    data.loc[data['sales_value'] > 30.0, ['item_id']] = 999999
    
    # 3. Придумайте свой фильтр
    # your_code
    
    # 4. Выбор топ-N самых популярных товаров (N = take_n_popular)
    # your_code
    
    popularity = data.groupby('item_id')['quantity'].sum().reset_index()
    popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
    top = popularity.sort_values('n_sold', ascending=False).head(take_n_popular).item_id.tolist()
    data.loc[~data['item_id'].isin(top), 'item_id'] = 999999
    return data

In [200]:
!mkdir src

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/4AHvzkRJtz1xtzd8TAY1TYy2ELuXFGVDpt4EL06wViQZghIhZcicUVA
Mounted at /content/drive


In [6]:
import os

In [7]:
cp /content/drive/'My Drive'/retail_train.csv /content

In [8]:
cp /content/drive/'My Drive'/product.csv /content

В корне проекта создайте модуль *src*. Положите функцию *prefilter_items* в файл *src/utils.py*  

Далее запустите код ниже и перейдите к заданию 2 (в конце ноутбука)

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix
from scipy.stats import mode
# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.bpr import BayesianPersonalizedRanking
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
#from src.metrics import precision_at_k, recall_at_k
#from src.utils import prefilter_items

In [10]:
data = pd.read_csv('retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

# Warm start
train_items = data_train['item_id'].unique()
train_users = data_train['user_id'].unique()

data_test = data_test[data_test['item_id'].isin(train_items)]
data_test = data_test[data_test['user_id'].isin(train_users)]

data_train.head(5)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [11]:
item_features = pd.read_csv('product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [12]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [13]:
n_items_before = data_train['item_id'].nunique()

data_train = prefilter_items(data_train, take_n_popular=5000)

n_items_after = data_train['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Decreased # items from 86865 to 5000


In [76]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count',
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

user_item_matrix.head(3)

item_id,202291,397896,420647,480014,731106,818980,818981,819063,819227,819255,819304,819308,819330,819518,819594,819643,819765,819840,819845,819927,819978,820082,820122,820165,820291,820301,820321,820347,820361,820486,820518,820560,820701,820895,821025,821083,821200,821209,821219,821344,...,13512017,13512060,13512542,13512965,13654702,13671759,13672065,13777104,13841744,13842088,13842090,13842214,13876901,13877192,13945141,13945244,13987135,14025185,14025461,14077333,14106445,15452677,15452812,15506577,15511891,15596279,15596488,15596515,15629778,15741861,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [79]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [16]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [17]:
%%time

model = AlternatingLeastSquares(factors=32, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


CPU times: user 464 ms, sys: 185 ms, total: 649 ms
Wall time: 1.85 s


In [18]:
model_bpr = BayesianPersonalizedRanking(factors=31, 
                                regularization=0.001,
                                iterations=15, 
                                num_threads=4)

model_bpr.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=True)

In [19]:
def get_recommendations(user, model, N=5):
    """Рекомендуем топ-N товаров"""
    
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]],  # !!! 
                                    recalculate_user=False)]
    return res

In [20]:
%%time
result['bm25_als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
result['bm25_bpr'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model_bpr, N=5))

CPU times: user 33.5 s, sys: 23.5 s, total: 57 s
Wall time: 29.1 s


In [21]:
result.apply(lambda row: precision_at_k(row['bm25_als'], row['actual']), axis=1).mean()

0.11562959333659902

In [22]:
result.apply(lambda row: precision_at_k(row['bm25_bpr'], row['actual']), axis=1).mean()

0.18196962273395176

In [186]:
top_purchases = data_train.groupby(['user_id', 'item_id'])['quantity'].count().reset_index()
top_purchases.sort_values('quantity', ascending=False, inplace=True)
top_purchases = top_purchases[top_purchases['item_id'] != 999999]

In [188]:
def get_similar_items_recommendation(user, model, top_purchases, N=5):
  """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""
  # Здесь top_purchases нужен извне. Далее он будет членом класса
  # Получаем топ собственных товаров юзера
  result = []
  own_items = top_purchases.loc[top_purchases['user_id'] == user]
  own_items.sort_values('quantity', ascending=False, inplace=True)
  own_items = own_items['item_id'].values[:N].tolist()
  #print(own_items)
  for own_item in own_items:
    similar_items = model.similar_items(itemid_to_id[own_item], N=2) # Надо предсказывать 2 ближайших, так как 1-й - это он сам
    #print(f'own: {own_item} nearest_to_own: {similar_items[1][0]}')
    #print(similar_items)
    result.append(id_to_itemid[similar_items[1][0]])
  return result

In [189]:
get_similar_items_recommendation(2375, model_bpr, top_purchases, N=5)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


[867931, 893018, 1044078, 844179, 896369]

In [190]:
# Кстати, bpr будет давать более схожие топы для похожих юзеров. У als топы похожих юзеров
# будут сильнее отличаться
# Надо доработать: исключить возможность дублей в итоговом

def get_similar_users_recommendation(user, model, N=5, K=5):
    """Рекомендуем топ-N товаров, среди купленных похожими K юзерами"""
    result = []
    # N=K+1, так как нулевой юзер - это тот, для которого ищем похожих
    nearests_users = model.similar_users(userid_to_id[user], N=K+1)
    #print(nearests_users)
    for i in range(1, K+1):
      recom = get_recommendations(id_to_userid[nearests_users[i][0]], model, N=N)
      result.append(recom)
      
    # Получаем матрицу, где по строкам - топ предсказаний для каждого из ближайших юзеров
    result = np.array(result)
    #print(result)
    # Для каждого столбца вычисляем моду 
    result = mode(result, axis = 0)

    return result[0][0].tolist()

In [191]:
get_similar_users_recommendation(2375, model_bpr, N=10, K=5)

[1082185,
 995242,
 981760,
 1029743,
 1004906,
 923746,
 6534178,
 1044078,
 859075,
 859075]

In [194]:
result['sim_item_rec'] = result['user_id'].apply(lambda x: get_similar_items_recommendation(x, model=model_bpr, top_purchases=top_purchases, N=5))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [70]:
result['sim_user_rec'] = result['user_id'].apply(lambda x: get_similar_users_recommendation(x, model=model_bpr, N=5, K=5))

In [72]:
result.apply(lambda row: precision_at_k(row['sim_user_rec'], row['actual']), axis=1).mean()

0.1788339049485525

In [195]:
result.apply(lambda row: precision_at_k(row['sim_item_rec'], row['actual']), axis=1).mean()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in long_scalars
  # This is added back by InteractiveShellApp.init_path()


0.05088495575221275

#### Создание словаря item_id_to_ctm

In [88]:
ctm = item_features.loc[item_features['brand'] == 'Private', ['item_id']]
data_train['is_ctm'] = data_train['item_id'].isin(ctm['item_id']).astype(int)
item_id = data_train['item_id'].values
is_ctm = data_train['is_ctm'].values
item_id_to_ctm = dict(zip(item_id, is_ctm))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [108]:
#item_id_to_ctm

In [109]:
# Проверка

In [ ]:
# data_train.iloc[100:]

In [117]:
10180324 in ctm['item_id'].values

True

In [118]:
item_id_to_ctm[10180324]

1

-----

# Домашнее задание

1. Перенесите метрики из ДЗ 1 src/metrics.py
3. Создайте модуль src/recommenders.py. Напищите код для класса ниже 
(задание обсуждали на вебинаре, для первой функции практически сделали) и положите его в src/recommenders.py
4. Проверьте, что все модули корректно импортируются

In [198]:
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight


class MainRecommender:
    """Рекоммендации, которые можно получить из ALS
    
    Input
    -----
    user_item_matrix: pd.DataFrame
        Матрица взаимодействий user-item
    """
    
    def __init__(self, data, items, weighting=True):
                
        # Топ покупок каждого юзера
        self.top_purchases = data.groupby(['user_id', 'item_id'])['quantity'].count().reset_index()
        self.top_purchases.sort_values('quantity', ascending=False, inplace=True)
        self.top_purchases = self.top_purchases[self.top_purchases['item_id'] != 999999]

        # Топ покупок по всему датасету
        self.overall_top_purchases = data.groupby('item_id')['quantity'].count().reset_index()
        self.overall_top_purchases.sort_values('quantity', ascending=False, inplace=True)
        self.overall_top_purchases = self.overall_top_purchases[self.overall_top_purchases['item_id'] != 999999]
        self.overall_top_purchases = self.overall_top_purchases.item_id.tolist()
        
        self.user_item_matrix = self.prepare_matrix(data)  # pd.DataFrame
        self.id_to_itemid, self.id_to_userid, self.itemid_to_id, self.userid_to_id = prepare_dicts(self.user_item_matrix)
        
        # Словарь {item_id: 0/1}. 0/1 - факт принадлежности товара к СТМ
        ctm = items.loc[items['brand'] == 'Private', ['item_id']]
        data['is_ctm'] = data['item_id'].isin(ctm['item_id']).astype(int)
        item_id = data_train['item_id'].values
        is_ctm = data_train['is_ctm'].values
        self.item_id_to_ctm = dict(zip(item_id, is_ctm))
        
        # Own recommender обучается до взвешивания матрицы
        self.own_recommender = self.fit_own_recommender(self.user_item_matrix)
        
        if weighting:
            self.user_item_matrix = bm25_weight(self.user_item_matrix.T).T 
        
        self.model = self.fit(self.user_item_matrix)
     
    @staticmethod
    def prepare_matrix(data):
        
        user_item_matrix = pd.pivot_table(data, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Нужно пробовать другие варианты
                                  aggfunc='count',
                                  fill_value=0
                                 )

        user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit
        
        return user_item_matrix
    
    @staticmethod
    def prepare_dicts(user_item_matrix):
        """Подготавливает вспомогательные словари"""
        
        userids = user_item_matrix.index.values
        itemids = user_item_matrix.columns.values

        matrix_userids = np.arange(len(userids))
        matrix_itemids = np.arange(len(itemids))

        id_to_itemid = dict(zip(matrix_itemids, itemids))
        id_to_userid = dict(zip(matrix_userids, userids))

        itemid_to_id = dict(zip(itemids, matrix_itemids))
        userid_to_id = dict(zip(userids, matrix_userids))
        
        return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id
     
    @staticmethod
    def fit_own_recommender(user_item_matrix):
        """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""
    
        own_recommender = ItemItemRecommender(K=1, num_threads=4)
        own_recommender.fit(csr_matrix(user_item_matrix).T.tocsr())
        
        return own_recommender
    
    @staticmethod
    def fit(user_item_matrix, n_factors=20, regularization=0.001, iterations=15, num_threads=4):
        """Обучает ALS"""
        
        model = AlternatingLeastSquares(factors=factors, 
                                             regularization=regularization,
                                             iterations=iterations,  
                                             num_threads=num_threads)
        model.fit(csr_matrix(self.user_item_matrix).T.tocsr())
        
        return model

    def get_similar_items_recommendation(self, user, filter_ctm=True, N=5):
        """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""
        result = []
        own_items = self.top_purchases.loc[self.top_purchases['user_id'] == user]
        own_items.sort_values('quantity', ascending=False, inplace=True)
        own_items = own_items['item_id'].values[:N].tolist()
        for own_item in own_items:
          similar_items = model.similar_items(itemid_to_id[own_item], N=2) # Надо предсказывать 2 ближайших, так как 1-й - это он сам
          result.append(id_to_itemid[similar_items[1][0]])
        # Не забывайте, что нужно учесть параметр filter_ctm

        return result
    
    def get_similar_users_recommendation(self, user, N=5):
        """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""
        result = []
        # N=K+1, так как нулевой юзер - это тот, для которого ищем похожих
        nearests_users = model.similar_users(userid_to_id[user], N=K+1)
        for i in range(1, K+1):
          recom = get_recommendations(id_to_userid[nearests_users[i][0]], model, N=N)
          result.append(recom)
      
        # Получаем матрицу, где по строкам - топ предсказаний для каждого из ближайших юзеров
        result = np.array(result)
        # Для каждого столбца вычисляем моду 
        result = mode(result, axis = 0)

        return result[0][0].tolist()

----

Проверка, что все работает

In [201]:
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender